In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import average_precision_score
from scipy import sparse

import textwrap

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash_table.Format import Format, Sign
import dash_table
import dash_table.FormatTemplate as FormatTemplate
import plotly.graph_objects as go
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

In [2]:
df = pd.read_excel('C:/Users/vchawda/Desktop/FY21/BOOST/Dataset/Final/Engagement Data_Final_FY20.xlsx')
df.head()

,Ultimate Parent Name,GUP Industry L1,GUP Industry L2,GUP Client Type,Program,Member Firm,Client Service L1,Client Service L2,Client Service L3,Client Service L4,GUP Industry L3,Year,Global LCSP,Net Service Revenue
0,3M Company,"Energy, Resources and Industrials","Oil, Gas and Chemicals",Advisory,Global Strategic,Africa,Tax and Legal,Global Employer Services,Global Mobility Compensation and Tax Compliance,-,Chemicals and Specialty Materials,2016,"WAELTER, ANTHONY",7501.949516
1,3M Company,"Energy, Resources and Industrials","Oil, Gas and Chemicals",Advisory,Global Strategic,Africa,Tax and Legal,Global Employer Services,Global Mobility Compensation and Tax Compliance,-,Chemicals and Specialty Materials,2017,"WAELTER, ANTHONY",5964.505820
2,3M Company,"Energy, Resources and Industrials","Oil, Gas and Chemicals",Advisory,Global Strategic,Africa,Tax and Legal,Global Employer Services,Global Mobility Compensation and Tax Compliance,-,Chemicals and Specialty Materials,2018,"WAELTER, ANTHONY",5455.682348
3,3M Company,"Energy, Resources and Industrials","Oil, Gas and Chemicals",Advisory,Global Strategic,Africa,Tax and Legal,Global Employer Services,Global Mobility Compensation and Tax Compliance,-,Chemicals and Specialty Materials,2019,"WAELTER, ANTHONY",10178.986687
4,3M Company,"Energy, Resources and Industrials","Oil, Gas and Chemicals",Advisory,Global Strategic,Africa,Tax and Legal,Global Employer Services,Global Mobility Compensation and Tax Compliance,-,Chemicals and Specialty Materials,2020,"WAELTER, ANTHONY",-3531.114400


In [3]:
top_clients = df[df['Year']==2020].groupby(['Ultimate Parent Name'])['Net Service Revenue'].sum().round(2)
top_clients = top_clients.to_frame().reset_index()
top_clients = top_clients[top_clients['Net Service Revenue']>10000000]
top_clients

,Ultimate Parent Name,Net Service Revenue
0,3M Company,2.566839e+07
1,A.P. Moller - Maersk A/S,9.932514e+07
2,ABB LTD.,1.148914e+08
3,ABN AMRO Group N.V.(Stichting Administratiekan...,5.572485e+07
5,AIA Group Limited,2.326963e+07
...,...,...
506,Wesfarmers Limited,1.673662e+07
507,Westpac Banking Corporation,1.339170e+07
509,"Wittington Investments, Limited (George Weston)",3.418878e+07
514,Zurich Insurance Group AG,3.022818e+07


In [4]:
# df['Client Service L3'].replace(' ', np.nan, inplace=True)
df['Client Service L3'].replace('-', np.nan, inplace=True)
df['GUP Industry L3'].replace(np.nan,'Unknown',inplace=True)
df.sort_values(by=['Ultimate Parent Name'], inplace=True)
df = df[df['Ultimate Parent Name'].isin(top_clients['Ultimate Parent Name'])]
df = df.astype({'Year': 'object'})
df = df[df['GUP Client Type'] != 'Audit']
df = df[df['Net Service Revenue']>0]
# df = df[df['Member Firm'] != 'Unknown']
df.dropna(subset=['Client Service L3'])
# # df['GUP Client Type'].replace(['A','R'],['Audit','Relationship'],inplace=True)
# # df.head()
print (df.shape)

(118110, 14)


In [5]:
df['GUP Industry L3'].unique()

array(['Chemicals and Specialty Materials', 'Transportation',
       'Industrial Products', 'Banking and Capital Markets (FS)',
       'Unknown', 'Energy, Utilities and Extractives',
       'Media and Entertainment', 'Telecom',
       'Hardware and Semiconductors',
       'Life, Health and Savings Insurance', 'Biopharma',
       'Aerospace and Defense', 'General Insurance',
       'Software and Services', 'Mining', 'Food and Beverage',
       'Center of Government', 'Original Equipment Manufacturers',
       'Med Tech', 'Agribusiness', 'Heavy Machinery and Equipment',
       'Oil and Gas', 'Mass and Discount Stores',
       'Personal and Household Goods', 'Power and Utilities (ERI)',
       'Industrial Conglomerates', 'Specialty and Department Stores',
       'Public Institutions', 'European Institutions',
       'Health Care Providers', 'Private Equity, Sovereign Wealth Funds',
       'Private Sector Development', 'Grocery and Convenience Stores',
       'Hospitality', 'Reinsurance', 

In [6]:
df['SA_Bool'] = 1
df.head()

,Ultimate Parent Name,GUP Industry L1,GUP Industry L2,GUP Client Type,Program,Member Firm,Client Service L1,Client Service L2,Client Service L3,Client Service L4,GUP Industry L3,Year,Global LCSP,Net Service Revenue,SA_Bool
0,3M Company,"Energy, Resources and Industrials","Oil, Gas and Chemicals",Advisory,Global Strategic,Africa,Tax and Legal,Global Employer Services,Global Mobility Compensation and Tax Compliance,-,Chemicals and Specialty Materials,2016,"WAELTER, ANTHONY",7501.949516,1
395,3M Company,"Energy, Resources and Industrials","Oil, Gas and Chemicals",Advisory,Global Strategic,North and South Europe,Tax and Legal,Transfer Pricing,Transfer Pricing Advisory,-,Chemicals and Specialty Materials,2016,"WAELTER, ANTHONY",42976.722665,1
396,3M Company,"Energy, Resources and Industrials","Oil, Gas and Chemicals",Advisory,Global Strategic,Portugal,Consulting,Core Business Operations,Systems Engineering,Service Delivery Optimization,Chemicals and Specialty Materials,2018,"WAELTER, ANTHONY",3001.349474,1
398,3M Company,"Energy, Resources and Industrials","Oil, Gas and Chemicals",Advisory,Global Strategic,Portugal,Tax and Legal,Business Tax,NaN,-,Chemicals and Specialty Materials,2017,"WAELTER, ANTHONY",12244.190103,1
399,3M Company,"Energy, Resources and Industrials","Oil, Gas and Chemicals",Advisory,Global Strategic,Portugal,Tax and Legal,Business Tax,NaN,-,Chemicals and Specialty Materials,2018,"WAELTER, ANTHONY",6022.601063,1


In [7]:
# FilteredDF = pd.pivot_table(df[(df['Member Firm'] == 'US')], values='SA_Bool', index='Ultimate Parent Name', columns='Client Service L3', aggfunc='sum').fillna(0)
# FilteredDF
service_revenue = pd.pivot_table(df, values='Net Service Revenue', 
                                 index=['Client Service L3','Client Service L2','Client Service L1'], aggfunc='mean').fillna(0).reset_index()
service_revenue.drop('Net Service Revenue', axis=1, inplace=True)
service_revenue

,Client Service L3,Client Service L2,Client Service L1
0,Accounting Operations Assurance,Assurance Services,Audit and Assurance
1,"Advertising, Marketing and Commerce",Customer and Marketing,Consulting
2,Analytics and Cognitive,"Strategy, Analytics and M and A",Consulting
3,Business Assurance,Assurance Services,Audit and Assurance
4,Business Process Technology,Business Process Solutions,Tax and Legal
...,...,...,...
91,Transfer Pricing Advisory,Transfer Pricing,Tax and Legal
92,Transfer Pricing Controversy,Transfer Pricing,Tax and Legal
93,Transfer Pricing Documentation,Transfer Pricing,Tax and Legal
94,Turnaround and Performance Improvement (includ...,Restructuring Services,Financial Advisory


In [8]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

# FilteredSets = FilteredDF.applymap(encode_units).reset_index()
# FilteredSets.head()

In [9]:
# Service_data = FilteredSets.drop('Ultimate Parent Name',1)
# del Service_data.columns.name
# Service_data.head()

In [10]:
# magnitude = np.sqrt(np.square(Service_data).sum(axis=1))
# normalize_data = Service_data.divide(magnitude, axis='index')
# normalize_data.head()

In [11]:
def calculate_similarity(dataframe):
    data_sparse = sparse.csr_matrix(dataframe)
    similarities = cosine_similarity(data_sparse.transpose())
    sim = pd.DataFrame(data=similarities, index= dataframe.columns, columns= dataframe.columns)
    return sim

# # Similarity matrix
# data_matrix = calculate_similarity(normalize_data)
# data_matrix.head()

In [12]:
# data_neighbours = pd.DataFrame(index=data_matrix.columns, columns=range(1,11))
# for i in range(0, len(data_matrix.columns)):
#     data_neighbours.iloc[i,:10] = data_matrix.iloc[0:,i].sort_values(ascending=False)[:10].index

# data_neighbours.head(50)

In [13]:
def get_clients(firm):
    clientdf = pd.pivot_table(df[(df['Member Firm'] == firm)], values='SA_Bool', index='Ultimate Parent Name', columns='Client Service L3', aggfunc='sum').fillna(0)
    return clientdf.index.unique()
        

In [14]:
def get_recommendation(selected_client,member_firm):
    client = selected_client
    
    FilteredDF = pd.pivot_table(df[(df['Member Firm'] == member_firm)], values='SA_Bool', index='Ultimate Parent Name', columns='Client Service L3', aggfunc='sum').fillna(0)
    FilteredSets = FilteredDF.applymap(encode_units).reset_index()
        
    Service_data = FilteredSets.drop('Ultimate Parent Name',1)
    del Service_data.columns.name
    magnitude = np.sqrt(np.square(Service_data).sum(axis=1))
    normalize_data = Service_data.divide(magnitude, axis='index')
    data_matrix = calculate_similarity(normalize_data)
    data_neighbours = pd.DataFrame(index=data_matrix.columns, columns=range(1,16))
    for i in range(0, len(data_matrix.columns)):
        data_neighbours.iloc[i,:15] = data_matrix.iloc[0:,i].sort_values(ascending=False)[:15].index
    
    client_index = FilteredSets[FilteredSets['Ultimate Parent Name'] == client].index.tolist()[0]
    client_services = normalize_data.loc[client_index]
    client_services = client_services[client_services >0].index.values
    
    similar_services = data_neighbours.loc[client_services]
    list_similar_services  = similar_services.values.tolist()
    list_similar_services  = list(set([item for sublist in list_similar_services for item in sublist]))
    
    neighbourhood = data_matrix[list_similar_services].loc[list_similar_services]
#     neighbourhood_unstack = neighbourhood.unstack().reset_index().round(2)
#     neighbourhood_unstack.columns=['Client Service L3 v1','Client Service L3 v2','Correlation']
    
#     client_vector = Service_data.loc[client_index].loc[list_similar_services]
#     weighted_score = neighbourhood.dot(client_vector).div(neighbourhood.sum(axis=1))
#     weighted_score = weighted_score.drop(client_services)
    clientneighbourhood = neighbourhood.loc[client_services].sum(axis=0)
    highest = clientneighbourhood.nlargest(1)[0]
#     recommendations_final = clientneighbourhood.drop(client_services).nlargest(100).div(highest)
    recommendations_final = clientneighbourhood.nlargest(50).div(highest)
    
    grp_yr = df[(df['Member Firm']== member_firm)&(df['Ultimate Parent Name']== client)].groupby(['Ultimate Parent Name','Client Service L3'])['Year'].max()
    # g.reset_index()
    lastpur = grp_yr.droplevel(0).to_frame().reset_index()
    
    client_vector = normalize_data.loc[client_index].loc[list_similar_services]
    client_vectordf = client_vector.to_frame().reset_index()
    client_vectordf.columns=['Client Service L3','Norm']
    client_lastpur = client_vectordf.merge(lastpur, how='left',on='Client Service L3')
    client_lastpur.fillna(0, inplace=True)
    client_lastpur['Last Purchased']=['Never / Before 2018' if 2021-x>3 else x for x in client_lastpur['Year']]
    client_lastpur.drop(['Norm','Year'], axis=1, inplace=True)
    
    recommendation = pd.DataFrame(recommendations_final.nlargest(50))
    recommendation.columns=['REC. Score']
    recommendation.index = recommendation.index.set_names(['Client Service L3'])
    
    
    service_revenue = pd.pivot_table(df[(df['Member Firm'] == member_firm)], values='Net Service Revenue', 
                                 index=['Client Service L3','Client Service L2','Client Service L1'], columns='Year', aggfunc='mean').fillna(0)
    service_revenue.reset_index(level=[1,2], inplace=True)
    service_revenue.drop(columns=[2016, 2017, 2018, 2019], axis=1, inplace=True)
    service_revenue.rename(columns={service_revenue.columns[2]:str(service_revenue.columns[2])+' Avg. NSR'}, inplace=True)

    recommend_output = recommendation.join(service_revenue, on='Client Service L3', how='left').reset_index().round(3)
    recommend_output = recommend_output.merge(client_lastpur, on='Client Service L3', how='left')
    cols = [recommend_output.columns[3], recommend_output.columns[2],recommend_output.columns[0],recommend_output.columns[5], recommend_output.columns[1], recommend_output.columns[4]]
    recommend_output = recommend_output[cols]
    
    return recommend_output

In [15]:
def get_correlation(firm, ind, sec, subsec):
    
    if firm==None:
        fil_firm = df['Member Firm'].notnull()
    else:
        fil_firm = df['Member Firm']==firm
    
    if ind==None:
        fil_ind = df['GUP Industry L1'].notnull()
    else:
        fil_ind = df['GUP Industry L1']==ind
        
    if sec==None:
        fil_sec = df['GUP Industry L2'].notnull()
    else:
        fil_sec = df['GUP Industry L2']==sec
        
    if subsec==None:
        fil_ssec = df['GUP Industry L3'].notnull()
    else:
        fil_ssec = df['GUP Industry L3']==subsec
        
    FilteredDF = pd.pivot_table(df[fil_firm & fil_ind & fil_sec & fil_ssec], values='SA_Bool', index='Ultimate Parent Name', columns='Client Service L3', aggfunc='sum').fillna(0)
    FilteredSets = FilteredDF.applymap(encode_units).reset_index()
        
    Service_data = FilteredSets.drop('Ultimate Parent Name',1)
    del Service_data.columns.name
    magnitude = np.sqrt(np.square(Service_data).sum(axis=1))
    normalize_data = Service_data.divide(magnitude, axis='index')
    corr_matrix = calculate_similarity(normalize_data)
    
    matrix_table = corr_matrix.unstack().to_frame().reset_index()
    matrix_table.columns = ['Client Service L3','Client Service L3 v2', 'Correlation']
    # matrix_table
   
    corr_table = matrix_table.merge(service_revenue, on='Client Service L3', how='left')
    corr_table = corr_table[['Client Service L1', 'Client Service L2', 'Client Service L3', 'Client Service L3 v2', 'Correlation']]
    
    return corr_matrix, corr_table

In [16]:
external_stylesheets = ['https://www.w3schools.com/w3css/4/w3.css']



app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
# app.css.append_css('https://codepen.io/chriddyp/pen/bWLwgP.css')

In [17]:
# client_detail = pd.pivot_table(df[(df['Member Firm'] == 'US')&(df['Ultimate Parent Name']=='AXA')], 
#                                    index=['Ultimate Parent Name', 'GUP Industry L2', 'GUP Client Type']).reset_index()
# client_detail.drop(['Net Service Revenue','SA_Bool'], axis=1, inplace=True)
# client_detail

In [18]:
def wrap(list):
    my_wrap = textwrap.TextWrapper(width = 30)
    wrap_list1 = [(my_wrap.fill(text = i)) for i in list]
    wrap_list2 = [(j.replace("\n", "<br>")) for j in wrap_list1]
    return wrap_list2

In [19]:
app.layout = html.Div([
    html.Div([
        html.Img(src='/assets/Deloitte NEW small 2.png',className='w3-display-topright',style={
                    'width':'120px','margin':'10px'
        }),
        
        html.H2([
            html.B(['Service Recommendation Dashboard'])
        
        ],style={'text-shadow':'1px 1px 0 #444','font-family': 'Open Sans','margin':'5px'})
    
    ], className='w3-container w3-black'),
    
    html.Div([
        html.H6(['Select a member firm:'], id='dd_title_mf',
                className='w3-container w3-col w3-margin-bottom w3-text-white',style={'width':'225px','font-family': 'Open Sans','margin-top':'20px'}),
        
        dcc.Dropdown(
            id='mf_dd',
            options=[
                {'label': i, 'value': i} for i in sorted(df['Member Firm'].unique())
            ],
            placeholder="Select a member firm..",
            value=None,className='w3-quarter w3-cell-middle w3-text-black',style={'margin-top':'5px'}),

        html.H6(['Select a client:'], id='dd_title_client',
                className='w3-container w3-col w3-margin-top w3-margin-bottom w3-text-white w3-right-align',style={'width':'200px','font-family': 'Open Sans'}),
        dcc.Dropdown(
            id='client_dd',
            options=[],
#             options=[
#                 {'label': i, 'value': i} for i in df['Ultimate Parent Name'].unique()
#             ],
            placeholder="Select a client..",
            value=None,className='w3-third w3-cell-middle w3-text-black',style={'margin-top':'5px'}),
                
        html.Button(id='submit-button', n_clicks=0, children='Submit',
                   className= ' w3-black w3-button w3-medium w3-round-large',style={'margin-left':'50px','margin-top':'10px'})
        
    ], className='w3-row',style={'font-family': 'Open Sans', 'font-size':'14px','line-height':'1','background-color':'#86bc25'}),
    
    html.Div([
        html.H5('Client details:',className='w3-row',style={'fontWeight':'bold','margin':'2px','margin-left':'16px'}),
        html.Div([
            dash_table.DataTable(
            id='client_table',
            style_cell={'padding': '5px','font-family':'Open Sans','font-size':'14px','line-height':'1'},
            style_header={
                        'backgroundColor': 'white',
                        'fontWeight': 'bold'
                    },
            style_cell_conditional=[
                    {
                            'if': {'column_id': col},
                            'textAlign': 'center'
                        } for col in ['Ultimate Parent Name','Program','GUP Industry L2', 'GUP Client Type', 'Global LCSP']

                    ]+[
                    {
                        'if': {'column_id': 'Ultimate Parent Name'},
                         'width': '20%'},
                    {
                        'if': {'column_id': 'Program'},
                         'width': '20%'},
                    {
                        'if': {'column_id': 'GUP Industry L2'},
                         'width': '20%'},
                    {
                        'if': {'column_id': 'GUP Client Type'},
                         'width': '20%'},
                    {
                        'if': {'column_id': 'Global LCSP'},
                         'width': '20%'}
                ]
            )
        ],className='w3-container')
#         html.Div(id='client_table')
    ],className='w3-row w3-light-grey',style={'height':'100px','font-family': 'Open Sans'}),
    
    html.Div([
            
        html.H5('Recommendations for Client:',className='w3-container w3-quarter',style={'fontWeight':'bold','font-family': 'Open Sans'}),
                
        html.H6('Client Service L1: ', className='w3-container w3-quarter w3-right-align w3-margin-top',
                style={'width':'300px','fontWeight':'bold','font-style':'italic','font-family': 'Open Sans'}),
        dcc.Dropdown(
            id='dd_bus',
            options=[
                {'label': i, 'value': i} for i in sorted(df['Client Service L1'].unique())
            ],
            multi=True,
            placeholder="Select a business..",
            value=[],className='w3-quarter w3-text-black',
            style={'margin-top':'5px','margin-right':'100px'}),

        html.H6('Last Purcahsed: ', className='w3-container w3-quarter w3-right-align',
                style={'width':'200px','fontWeight':'bold','font-style':'italic', 'margin-top':'10px','font-family': 'Open Sans'}),
        dcc.RadioItems(
            id='ri_lastpur',
            options=[
                {'label': 'Never / Before 2018', 'value': 'Never / Before 2018'},
                {'label': 'Anytime ', 'value': ''}
            ],
            value='Never / Before 2018', className='w3-rest w3-margin-top', 
            style={'width':'250px'}, labelStyle={'font-size':'14px', 'padding-top':'5px'}, 
            inputStyle={'padding':'10px','margin-right':'5px', 'margin-left':'5px'})

    ], className='w3-row',style={'margin-bottom':'10px','font-family': 'Open Sans'}),
    
    html.Div([
        
        dash_table.DataTable(
            id='recommendation_table',
            editable=False,
            style_as_list_view=True,
            style_cell={'padding': '5px', 'font-family':'Open Sans','font-size':'14px'},
            style_header={
                    'backgroundColor': 'white',
                    'fontWeight': 'bold'
                },
            style_table={
                    'maxHeight': '65vh',
                    'overflowY': 'scroll',
            },
            style_cell_conditional=[
                {
                        'if': {'column_id': c},
                        'textAlign': 'left'
                    } for c in ['Client Service L1','Client Service L2', 'Client Service L3','Last Purchased']
                
                ]+[
                {
                    'if': {'column_id': 'Client Service L1'},
                     'width': '15%'},
                {
                    'if': {'column_id': 'Client Service L2'},
                     'width': '20%'},
                {
                    'if': {'column_id': 'Client Service L3'},
                     'width': '30%'},
                {
                    'if': {'column_id': 'Last Purchased'},
                     'width': '15%'},
                {
                    'if': {'column_id': 'REC. Score'},
                     'width': '5%'},
                {
                    'if': {'column_id': '2020 Avg. NSR'},
                     'width': '15%'}
            ],
            style_data_conditional=[
                {
                    'if': {
                        'column_id': 'REC. Score',
                        'filter_query': '{REC. Score} >= 0'
                    },
                    'backgroundColor': '#E3E48D',
                    'color': '#E3E48D',
                    
                },
                {
                    'if': {
                        'column_id': 'REC. Score',
                        'filter_query': '{REC. Score} >= 0.5'
                    },
                    'backgroundColor': '#6FC2B4',
                    'color': '#6FC2B4',
                },
                {
                    'if': {
                        'column_id': 'REC. Score',
                        'filter_query': '{REC. Score} >= 0.65'
                    },
                    'backgroundColor': '#0097A9',
                    'color': '#0097A9',
                },
                {
                    'if': {
                        'column_id': 'REC. Score',
                        'filter_query': '{REC. Score} >= 0.8'
                    },
                    'backgroundColor': '#005587',
                    'color': '#005587',
                }
            ]
            
        ),
        html.Hr(id='ruler',style={'border': '2.5px solid #86bc25'}),
        html.H4('Services Correlation:',className='w3-margin-top', style={'fontWeight':'bold'}),
        html.Div([
                
            html.H5(['Filters:'], id='hd_filters', className='w3-text-white w3-padding w3-row', 
                    style={'margin-top':'0px','background-color':'#86bc25'}),
            
            html.H6(['Member Firm:'], id='hd_firm', className='w3-text-white w3-margin-left',
                    style={'width':'200px', 'fontStyle':'italic', 'margin-top':'35px'}),
            dcc.Dropdown(
                    id='dd_firm',
                    options=[
                        {'label': i, 'value': i} for i in sorted(df['Member Firm'].unique())
                    ],
                    placeholder="Select a member firm..",
                    value=None,className='w3-cell-middle w3-margin-left w3-text-black',
                    style={'width':'240px','font-family':'Open Sans','font-size':'14px','line-height':'1'}),
            
            html.H6(['Industry:'], id='hd_ind', className='w3-text-white w3-margin-left', 
                    style={'width':'200px', 'fontStyle':'italic', 'margin-top':'35px'}),
            dcc.Dropdown(
                    id='dd_ind',
                    options=[
                        {'label': i, 'value': i} for i in sorted(df['GUP Industry L1'].unique())
                    ],
                    placeholder="Select an industry..",
                    value=None,className='w3-cell-middle w3-margin-left w3-text-black',
                    style={'width':'240px','font-family':'Open Sans','font-size':'14px','line-height':'1'}),
            
            html.H6(['Sector:'], id='hd_sect', className='w3-text-white w3-margin-left', 
                    style={'width':'200px', 'fontStyle':'italic', 'margin-top':'35px'}),
            dcc.Dropdown(
                    id='dd_sect',
                    options=[
                        {'label': i, 'value': i} for i in sorted(df['GUP Industry L2'].unique())
                    ],
                    placeholder="Select a sector..",
                    value=None,className='w3-cell-middle w3-margin-left w3-text-black',
                    style={'width':'240px','font-family':'Open Sans','font-size':'14px','line-height':'1'}),
            
            html.H6(['Sub-Sector:'], id='hd_ssect', className='w3-text-white w3-margin-left', 
                    style={'width':'200px', 'fontStyle':'italic', 'margin-top':'35px'}),
            dcc.Dropdown(
                    id='dd_ssect',
                    options=[
                        {'label': x, 'value': x} for x in sorted(df['GUP Industry L3'].unique())
                    ],
                    placeholder="Select a sub-sector..",
                    value=None,className='w3-cell-middle w3-margin-left w3-text-black',
                    style={'width':'240px','font-family':'Open Sans','font-size':'14px','line-height':'1'}),
            
            html.H6(['Client Service L1:'], id='hd_bus', className='w3-text-white w3-margin-left', 
                    style={'width':'200px', 'fontStyle':'italic', 'margin-top':'35px'}),
            dcc.Dropdown(
                    id='dd_bus_corr',
                    options=[
                        {'label': i, 'value': i} for i in sorted(df['Client Service L1'].unique())
                    ],
                    placeholder="Select a business..",
                    value=None,className='w3-cell-middle w3-margin-left w3-text-black',
                    style={'width':'240px','font-family':'Open Sans','font-size':'14px','line-height':'1'})
            
        
            ],className='w3-quarter w3-black w3-margin-right', 
            style={
                    'height':'90vh',
                    'max-width':'275px'
        }),
        html.Div([
            dcc.Tabs([
                dcc.Tab(label='Chart', children=[
                    dcc.Graph(
                        id='corr-matrix',
                        style={
                            'max-height':'86vh'
                        }
                    )
                ], style={
                    'padding-top': '10px'
                }, selected_style={
                    'padding-top': '10px',
                    'font-weight': 'bold'
                }),
                dcc.Tab(label='Table', children=[
                    dash_table.DataTable(
                    id='corr-table',
                    editable=False,
                    style_as_list_view=True,
                    style_cell={'padding': '2px','font-family':'Open Sans', 'font-size':'13px'},
                    style_header={
                                'backgroundColor': 'white',
                                'fontWeight': 'bold'
                            },
                    style_table={
                            'maxHeight': '600px',
                            'overflowY': 'scroll',
                    },
                    style_cell_conditional=[
                    {
                            'if': {'column_id': c},
                            'textAlign': 'left'
                        } for c in ['Client Service L1','Client Service L2','Client Service L3','Client Service L3 v2']

                    ]+[
                    {
                        'if': {'column_id': 'Client Service L1'},
                         'width': '25%'},
                    {
                        'if': {'column_id': 'Client Service L2'},
                         'width': '25%'},
                    {
                        'if': {'column_id': 'Client Service L3'},
                         'width': '35%'},
                    {
                        'if': {'column_id': 'Client Service L3 v2'},
                         'width': '35%'},
                    {
                        'if': {'column_id': 'Correlation'},
                         'width': '5%'}
                    ],
                    style_data_conditional=[
                        {
                            'if': {
                                'column_id': 'Correlation',
                                'filter_query': '{Correlation} >= 0'
                            },
                            'backgroundColor': '#E3E48D',
                            'color': '#E3E48D',

                        },
                        {
                            'if': {
                                'column_id': 'Correlation',
                                'filter_query': '{Correlation} >= 0.5'
                            },
                            'backgroundColor': '#6FC2B4',
                            'color': '#6FC2B4',
                        },
                        {
                            'if': {
                                'column_id': 'Correlation',
                                'filter_query': '{Correlation} >= 0.65'
                            },
                            'backgroundColor': '#0097A9',
                            'color': '#0097A9',
                        },
                        {
                            'if': {
                                'column_id': 'Correlation',
                                'filter_query': '{Correlation} >= 0.8'
                            },
                            'backgroundColor': '#005587',
                            'color': '#005587',
                        }
                    ])
                ], style={
                    'padding-top': '10px'
                }, selected_style={
                    'padding-top': '10px',
                    'font-weight': 'bold'
                })
            ], colors={
                'border': 'gainsboro',
                'primary': 'yellowgreen',
                'background': 'white'
            }, style={
                'height': '44px'
            })
        
        ], className='w3-rest')
        
    ], className='w3-container',style={'font-family':'Open Sans'})

],style={ 'font-family':'Open Sans'})

@app.callback([
    Output('client_dd','options'),
    Output('dd_firm','value')
], [Input('mf_dd','value')])
def populate_firms(mf):
    client_list = get_clients(mf)
    options=[{'label': l, 'value': l} for l in client_list]
    return [options, mf]


@app.callback([
    Output('corr-matrix', 'figure'),
    Output('corr-table', 'columns'),
    Output('corr-table', 'data')
], [Input('dd_firm', 'value'),
    Input('dd_ind','value'),
    Input('dd_sect','value'),
    Input('dd_ssect','value'),
    Input('dd_bus_corr','value')])
def compute_corr(firm_val, ind_val, sect_val, ssect_val, corrbus_val):
# def compute_corr(firm_val, ind_val, sect_val, corrbus_val):
    if firm_val is None:
        raise PreventUpdate
        
    bus = corrbus_val
    corr_matrix, corr_table = get_correlation(firm_val, ind_val, sect_val, ssect_val)
    
    if bus==None:
        fil_bus = corr_table['Client Service L1'].notnull()
    else:
        fil_bus = corr_table['Client Service L1']==bus
    
    corr_table = corr_table[fil_bus].round(2)
    columns_corr=[{"name": i, "id": i} for i in corr_table.columns]
    data_corr=corr_table.reset_index().to_dict('records')
    
    figure={
            'data':[go.Heatmap(
                       z=corr_matrix.values,
                       x=corr_matrix.index,
                       y=corr_matrix.columns,
                       colorbar=dict(ticks = ''),
                       hoverongaps = False,
                       colorscale='YlGnBu'
                       )],
            'layout': dict(title = dict(text='Correlation Matrix',font=dict(family='Open Sans', size=20, weight='bold')),
                        xaxis= dict(title= 'Client Service L3'),
                        yaxis= dict(tickangle=-20),
                        font=dict(family='Open Sans', size=10, weight='bold'),
                        margin=dict(l=200,t=60,b=140,r=30)
                    )
            }
    return figure, columns_corr, data_corr


@app.callback([
    Output('recommendation_table', 'columns'),
    Output('recommendation_table', 'data'),
    Output('client_table', 'columns'),
    Output('client_table', 'data')
], [Input('submit-button', 'n_clicks'),
    Input('ri_lastpur','value'),
    Input('dd_bus','value')
], [State('client_dd', 'value'),
    State('mf_dd', 'value')])
def compute_output(clicks, last_pur, business, client_value, mf_value):
    if client_value is None:
        raise PreventUpdate
    
    client_detail = pd.pivot_table(df[(df['Ultimate Parent Name']==client_value)&(df['Member Firm']==mf_value)], 
                                   index=['Ultimate Parent Name', 'Program', 'GUP Industry L2', 'GUP Client Type', 'Global LCSP']).reset_index()
    client_detail.drop(["Net Service Revenue","SA_Bool"], axis=1, inplace=True)
    columns_client=[{"name": i, "id": i} for i in client_detail.columns]
    data_client=client_detail.to_dict('records')
    
    recommendation = get_recommendation(client_value, mf_value)
    
    if last_pur=="":
        fil_lastpur = recommendation['Last Purchased'].notnull()
    else:
        fil_lastpur = recommendation['Last Purchased']==last_pur
        
    if business==[]:
        fil_bus = recommendation['Client Service L1'].notnull()
    else:
        fil_bus = recommendation['Client Service L1'].isin(business)
    
            
    filtered_rec = recommendation[fil_lastpur & fil_bus]
#     columns_rec=[{"name": j, "id": j} for j in recommendation.columns]
    columns_rec = [{
            'id': recommendation.columns[0],
            'name': recommendation.columns[0],
            'type': 'text'
        }, {
            'id': recommendation.columns[1],
            'name': recommendation.columns[1],
            'type': 'text',
        }, {
            'id': recommendation.columns[2],
            'name': recommendation.columns[2],
            'type': 'text',
        }, {
            'id': recommendation.columns[3],
            'name': recommendation.columns[3],
            'type': 'text',
        }, {
            'id': recommendation.columns[4],
            'name': recommendation.columns[4],
            'type': 'numeric',
            'format':FormatTemplate.percentage(1).sign(Sign.positive)
        }, {
            'id': recommendation.columns[5],
            'name': recommendation.columns[5],
            'type': 'numeric',
            'format': FormatTemplate.money(0)
        }]
    data_rec=filtered_rec.to_dict('records')
    
    
#     output = [client_value]
    
    return columns_rec, data_rec, columns_client, data_client

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Oct/2021 17:35:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 17:35:40] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 17:35:40] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 17:35:40] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [28/Oct/2021 17:35:40] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [28/Oct/2021 17:35:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 17:35:40] "GET /assets/Deloitte%20NEW%20small%202.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Oct/2021 17:35:40] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [28/Oct/2021 17:36:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 17:36:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 17:36:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Oct/2021 17:37:23] "POST /_dash-up